In [1]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np

In [2]:
DATA_DIR = "../data/"
TRAIN_IMG = DATA_DIR + "train/"
VAL_IMG = DATA_DIR + "validation/"
TEST_IMG = DATA_DIR + "test/"

In [3]:
names = pd.read_csv(DATA_DIR + 'class-descriptions-boxable.csv', names=['id','label'], usecols=[0,1])
names.head()

,id,label
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


In [4]:
keep = ['Human face', 'Man', 'Woman', 'Clothing', 'Car', 'Wheel', 'Food', 'Vehicle', 'Boy', 'Girl']
choosen_id = list(names[[i in keep for i in names['label']]]['id'])

In [5]:
train = pd.read_csv(DATA_DIR + "oidv6-train-annotations-bbox.csv")
train_present = set(os.listdir(TRAIN_IMG))
present = [im_id + '.jpg' in train_present for im_id in tqdm(train['ImageID'])]
train = train[present]
need = ['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax']
train = train[need]
need = [i in choosen_id for i in train['LabelName']]
train = train[need]
print(len(train['ImageID'].unique()))

100%|██████████| 14610229/14610229 [00:04<00:00, 3279296.88it/s]
100000


In [6]:
x_min, y_min = np.array(train['XMin']), np.array(train['YMin'])
x_max, y_max = np.array(train['XMax']), np.array(train['YMax'])
centerx = (x_max + x_min) / 2
centery = (y_max + y_min) / 2
width = x_max - x_min
height = y_max - y_min
train['CentroidX'] = centerx
train['CentroidY'] = centery
train['Width'] = width
train['Height'] = height
need = ['ImageID', 'LabelName', 'CentroidX', 'CentroidY', 'Width', 'Height']
train = train[need]
train.to_csv('cleaned_train_bbox.csv', index=False)

In [7]:
test = pd.read_csv(DATA_DIR + "test-annotations-bbox.csv")
test_present = set(os.listdir(TEST_IMG))
present = [im_id + '.jpg' in test_present for im_id in tqdm(test['ImageID'])]
test = test[present]
need = ['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax']
test = test[need]
need = [i in choosen_id for i in test['LabelName']]
test = test[need]
print(len(test['ImageID'].unique()))

100%|██████████| 937327/937327 [00:00<00:00, 3201926.63it/s]
10000


In [8]:
x_min, y_min = np.array(test['XMin']), np.array(test['YMin'])
x_max, y_max = np.array(test['XMax']), np.array(test['YMax'])
centerx = (x_max + x_min) / 2
centery = (y_max + y_min) / 2
width = x_max - x_min
height = y_max - y_min
test['CentroidX'] = centerx
test['CentroidY'] = centery
test['Width'] = width
test['Height'] = height
need = ['ImageID', 'LabelName', 'CentroidX', 'CentroidY', 'Width', 'Height']
test = test[need]
test.to_csv('cleaned_test_bbox.csv', index=False)

In [9]:
val = pd.read_csv(DATA_DIR + "validation-annotations-bbox.csv")
val_present = set(os.listdir(VAL_IMG))
present = [im_id + '.jpg' in val_present for im_id in tqdm(val['ImageID'])]
val = val[present]
need = ['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax']
val = val[need]
need = [i in choosen_id for i in val['LabelName']]
val = val[need]
print(len(val['ImageID'].unique()))

100%|██████████| 303980/303980 [00:00<00:00, 2714760.45it/s]5000



In [10]:
x_min, y_min = np.array(val['XMin']), np.array(val['YMin'])
x_max, y_max = np.array(val['XMax']), np.array(val['YMax'])
centerx = (x_max + x_min) / 2
centery = (y_max + y_min) / 2
width = x_max - x_min
height = y_max - y_min
val['CentroidX'] = centerx
val['CentroidY'] = centery
val['Width'] = width
val['Height'] = height
need = ['ImageID', 'LabelName', 'CentroidX', 'CentroidY', 'Width', 'Height']
val = val[need]
val.to_csv('cleaned_validation_bbox.csv', index=False)